# Note book to run colmap

NOTE: containers/pods for Colmap and neuralangelo are seperate as they have conflicting dependencies.

In [ ]:
# Imports
import gdown
import os
import validators

### Data:

Download the data from drive and update video path variable.

In [ ]:
#Download the video from drive:

#URl from google drive.
VIDEO_URL = ''

if validators.url(VIDEO_URL):
    print("Valid URL")
    !gdown --fuzzy {VIDEO_URL}
else:
    print("Invalid URL")


In [ ]:
# File path: --> copy the path to which the video was downloaded
video_path = ''

## COLMAP processing

1. Populate variables
2. Execute colmap command 

In [ ]:
# Variables:
    # SEQUENCE: your custom name for the video sequence.
    # PATH_TO_VIDEO: absolute/relative path to your video.
    # DOWNSAMPLE_RATE: temporal downsampling rate of video sequence (for extracting video frames).
    # SCENE_TYPE: can be one of  {outdoor,indoor,object}.

SEQUENCE = '' # Give a relevant name. Eg: Leg_AFO_30thMarch24, bottle_rig_15thMarch25.
PATH_TO_VIDEO = video_path
DOWNSAMPLE_RATE = '2' #Higher the downsample rate, lower number of frames extracted.
SCENE_TYPE = 'object' # currently most of our experiments are objects.

# Run colmap:

In [ ]:
# FFMPEG:
if SEQUENCE == '': # To avoid weird path creation
    print('[INFO] Setting sequence name to default value.')
    SEQUENCE = 'video_sequence'

data_path=f'datasets/{SEQUENCE}_ds{DOWNSAMPLE_RATE}'
print(f'[INFO] File path: {PATH_TO_VIDEO}')
if os.path.exists(PATH_TO_VIDEO):
    !chmod +x ../projects/neuralangelo/scripts/run_ffmpeg.sh
    !bash ../projects/neuralangelo/scripts/run_ffmpeg.sh {SEQUENCE} {PATH_TO_VIDEO} {DOWNSAMPLE_RATE}
else:
    print(f'[ERROR]: Following video path does not exist {PATH_TO_VIDEO}')


In [ ]:
# Run COLMAP:
print(f'[INFO]: Path to dataset: {data_path}')
!bash ../projects/neuralangelo/scripts/run_colmap.sh {data_path}

In [ ]:
# Generate transforms.json
!pip install numpy #Common error seen. - requires a pip install of numpy. 

#Git init submodule -> colmap repo:
!git submodule update --init --recursive

# One of the two commands works depending on machine:
# RUNPOD behaviour noticed: In runpod it picks up a different version of python and does not find compatible numpy. 
try:
    !python ../projects/neuralangelo/scripts/convert_data_to_json.py --data_dir {data_path} --scene_type {SCENE_TYPE}
except Exception as e:
    !python3.11 ../projects/neuralangelo/scripts/convert_data_to_json.py --data_dir {data_path} --scene_type {SCENE_TYPE}


### Send Data: RunpodCTL


!runpodctl send {data_path}